In [ ]:
import pyarrow.parquet as pa
import pandas as pd
import os 
table  = pa.read_table("/home/vulamanh/Documents/GRAPHRAG_DATN/src/data/synthetic/cross_queries.parquet")
df = table.to_pandas()


def remove_if_exist(file):
    if os.path.exists(file):
        os.remove(file)

In [2]:
from tqdm import tqdm
def get_text_chunks(df: pd.DataFrame, n_queries = 20):
    # Lấy hết câu hỏi khác nhau
    all_distinct_queries = df["query"].unique()

    # Obtain the small subset of questions (20 first questions)
    small_queries_list = all_distinct_queries[:n_queries]

    # Take the all chunks related to that subset of questions
    df_small = df[df["query"].isin(small_queries_list)]
    list_contexts = df_small["pos"].to_list()
    list_contexts = list(set([a[0] for a in list_contexts]))

    #Build the dictionary with key is the query and value is list of indexes of relevant chunks
    true_pos = {}
    map_contexts_to_id = {contexts: i for i, contexts in enumerate(list_contexts)}
    for query in tqdm(small_queries_list):
        pos_list = []
        df_query_specific = df[df["query"] == query]
        df_query_specific.reset_index(inplace = True)
        for i in range(len(df_query_specific)):
            pos_list.append(map_contexts_to_id[df_query_specific["pos"][i][0]])
        true_pos[query] = list(set(pos_list))

    return list_contexts, true_pos

In [3]:
list_context, true_pos = get_text_chunks(df, n_queries=50)

100%|██████████| 50/50 [00:06<00:00,  7.51it/s]


In [ ]:
import nest_asyncio
from nano_graphrag import GraphRAG, QueryParam
import json
import shutil
import numpy as np
from nano_graphrag import GraphRAG, QueryParam
from nano_graphrag._utils import wrap_embedding_func_with_attrs
import os
import pandas as pd
nest_asyncio.apply()
WORKING_DIR = "tests_80B"

remove_if_exist(f"{WORKING_DIR}/milvus_lite.db")
remove_if_exist(f"{WORKING_DIR}/kv_store_full_docs.json")
remove_if_exist(f"{WORKING_DIR}/kv_store_text_chunks.json")
remove_if_exist(f"{WORKING_DIR}/kv_store_community_reports.json")
remove_if_exist(f"{WORKING_DIR}/graph_chunk_entity_relation.graphml")
rag = GraphRAG(
        working_dir=WORKING_DIR,
        enable_naive_rag=True,
        enable_local= True
)


In [5]:
# rag.insert(list_context)

In [ ]:
# import nest_asyncio
# from nano_graphrag import GraphRAG, QueryParam
# import json
# import shutil
# import numpy as np
# from nano_graphrag import GraphRAG, QueryParam
# from nano_graphrag._utils import wrap_embedding_func_with_attrs
# import os
# import pandas as pd
# nest_asyncio.apply()
# WORKING_DIR = "tests"
# rag = GraphRAG(
#         working_dir=WORKING_DIR,
#         enable_naive_rag=True,
#         enable_local= True
# )

/home/vulamanh/miniconda3/envs/graphrag_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:nano-graphrag:Load KV full_docs with 40 data
INFO:nano-graphrag:Load KV text_chunks with 40 data
INFO:nano-graphrag:Load KV llm_response_cache with 323 data
INFO:nano-graphrag:Load KV community_reports with 0 data
INFO:nano-graphrag:Loaded graph from tests/graph_chunk_entity_relation.graphml with 471 nodes, 343 edges
INFO:nano-vectordb:Load (470, 1024) data
INFO:nano-vectordb:Init {'embedding_dim': 1024, 'metric': 'cosine', 'storage_file': 'tests/vdb_entities.json'} 470 data
INFO:nano-vectordb:Load (343, 1024) data
INFO:nano-vectordb:Init {'embedding_dim': 1024, 'metric': 'cosine', 'storage_file': 'tests/vdb_edges.json'} 343 data
INFO:nano-vectordb:Load (40, 1024) data
INFO:nano-vectordb:Init {'embedding_dim

In [7]:
# import networkx as nx
# from pyvis.network import Network

# # Load GraphML file
# graphml_file = "/home/vulamanh/Documents/GRAPHRAG_DATN/src/tests/graph_chunk_entity_relation.graphml"
# G = nx.read_graphml(graphml_file)

# # Initialize PyVis network
# net = Network(height="100vh", width="100vw", bgcolor="black", font_color="white", notebook = True)

# # Convert NetworkX graph to PyVis
# net.from_nx(G)

# # Improve Layout
# net.force_atlas_2based()
# net.toggle_physics(True)

# # Save and show graph
# net.show("graph.html")

# import webbrowser
# webbrowser.open('graph.html')

In [8]:
list_queries = list(true_pos.keys())
from tqdm import tqdm

In [9]:
# import nest_asyncio
# nest_asyncio.apply()
for query in tqdm(list_queries):
    result = rag.query(query, param=QueryParam(mode="local", top_k = 10))



  0%|          | 0/50 [00:00<?, ?it/s]

Directory 'tests/local_query' created.
Bộ Giao thông vận tải quy định các đơn vị kinh doanh vận tải nào phải lắp đặt thiết bị giám sát hành trình trên xe ô tô?
['"SỐ LƯỢNG PHƯƠNG TIỆN VẬN TẢI THUỘC ĐỐI TƯỢNG PHẢI LẮP ĐẶT THIẾT BỊ GIÁM SÁT HÀNH TRÌNH"', '"THIẾT BỊ GIÁM SÁT HÀNH TRÌNH"', '"ARTICLE 2"', '"ĐƠN VỊ CUNG CẤP DỊCH VỤ GIÁM SÁT HÀNH TRÌNH"', '"BỘ GIAO THÔNG VẬN TẢI"', '"TRANSPORT BUSINESSES"', '"HỆ THỐNG THÔNG TIN TỪ THIẾT BỊ GIÁM SÁT HÀNH TRÌNH"', '"AGENCIES, ORGANIZATIONS AND INDIVIDUALS INVOLVED IN PROVISION, MANAGEMENT AND USE OF DATA FROM VEHICLE TRACKING DEVICES NATIONWIDE"', '"ĐƠN VỊ KINH DOANH VẬN TẢI"', '"VEHICLE TRACKING SERVICE PROVIDERS AND AGENCIES"']


INFO:nano-graphrag:Using 10 entites, 0 communities, 45 relations, 11 text units
  2%|▏         | 1/50 [00:00<00:45,  1.08it/s]

Data has been written to output.json
Directory 'tests/local_query' created.
Bộ Giao thông vận tải giải thích như thế nào về các từ ngữ như 'hệ thống thông tin' và 'dữ liệu' trong quy định về thiết bị giám sát hành trình?
['"HỆ THỐNG THÔNG TIN"', '"HỆ THỐNG THÔNG TIN TỪ THIẾT BỊ GIÁM SÁT HÀNH TRÌNH ĐƯỢC SỬ DỤNG, CẬP NHẬT, KHAI THÁC VÀ QUẢN LÝ"', '"HỆ THỐNG THÔNG TIN TỪ THIẾT BỊ GIÁM SÁT HÀNH TRÌNH"', '"DỮ LIỆU"', '"HƯỚNG DẪN SỬ DỤNG, CẬP NHẬT, KHAI THÁC VÀ QUẢN LÝ HỆ THỐNG THÔNG TIN TỪ THIẾT BỊ GIÁM SÁT HÀNH TRÌNH"', '"QUẢN LÝ VÀ SỬ DỤNG DỮ LIỆU TỪ THIẾT BỊ GIÁM SÁT HÀNH TRÌNH"', '"SỐ LƯỢNG PHƯƠNG TIỆN VẬN TẢI THUỘC ĐỐI TƯỢNG PHẢI LẮP ĐẶT THIẾT BỊ GIÁM SÁT HÀNH TRÌNH"', '"DỮ LIỆU GIÁM SÁT HÀNH TRÌNH"', '"BỘ GIAO THÔNG VẬN TẢI"', '"DỮ LIỆU TỪ THIẾT BỊ GIÁM SÁT HÀNH TRÌNH"']


INFO:nano-graphrag:Using 10 entites, 0 communities, 20 relations, 8 text units
  4%|▍         | 2/50 [00:01<00:29,  1.62it/s]

...

Data has been written to output.json
Directory 'tests/local_query' created.
Bộ Giao thông vận tải quy định như thế nào về trách nhiệm của các đơn vị kinh doanh vận tải trong việc cung cấp dữ liệu từ thiết bị giám sát hành trình?
['"TRANSPORT BUSINESSES"', '"BỘ GIAO THÔNG VẬN TẢI"', '"SỐ LƯỢNG PHƯƠNG TIỆN VẬN TẢI THUỘC ĐỐI TƯỢNG PHẢI LẮP ĐẶT THIẾT BỊ GIÁM SÁT HÀNH TRÌNH"', '"ĐƠN VỊ CUNG CẤP DỊCH VỤ GIÁM SÁT HÀNH TRÌNH"', '"DIRECTORATE FOR ROADS OF VIETNAM"', '"DEPARTMENT OF TRANSPORT"', '"ARTICLE 2"', '"TỔNG CỤC ĐƯỜNG BỘ VIỆT NAM"', '"SỞ GIAO THÔNG VẬN TẢI"', '"AGENCIES, ORGANIZATIONS AND INDIVIDUALS INVOLVED IN PROVISION, MANAGEMENT AND USE OF DATA FROM VEHICLE TRACKING DEVICES NATIONWIDE"']


INFO:nano-graphrag:Using 10 entites, 0 communities, 117 relations, 11 text units
 98%|█████████▊| 49/50 [00:20<00:00,  2.41it/s]

Data has been written to output.json
Directory 'tests/local_query' created.
Pháp luật quy định các đơn vị kinh doanh vận tải phải đảm bảo tính chính xác của dữ liệu như thế nào?
['"TRANSPORT BUSINESSES"', '"DEPARTMENT OF TRANSPORT"', '"ĐƠN VỊ KINH DOANH VẬN TẢI"', '"TRANSPORT SERVICE PROVIDERS"', '"ĐƠN VỊ KINH DOANH VẬN TẢI TRONG VIỆC CHẤP HÀNH CÁC QUY ĐỊNH VỀ QUẢN LÝ HOẠT ĐỘNG VẬN TẢI"', '"SỐ GIAO THÔNG VẬN TẢI"', '"ARTICLE 2"', '"PROVINCIAL DEPARTMENTS OF TRANSPORT"', '"TRANSPORT BUSINESS"', '"DIRECTORATE FOR ROADS OF VIETNAM"']


INFO:nano-graphrag:Using 10 entites, 0 communities, 133 relations, 12 text units
100%|██████████| 50/50 [00:20<00:00,  2.42it/s]

Data has been written to output.json


In [10]:
import json
with open("/home/vulamanh/Documents/GRAPHRAG_DATN/src/tests/local_query/chosen_text_chunks.json", "r") as f:
    prediction = json.load(f)

In [28]:
prediction_id = {}
for k, v in prediction.items():
    ids = []
    for text_chunk in v:
        ids.append(list_context.index(text_chunk["content"]))
    
    prediction_id[k] = ids
    

In [ ]:
# original 
from graphRAG.storage.chroma_db import average_four_metrics
list_actual = list(true_pos.values())
list_predicted = list(prediction_id.values())
average_four_metrics(list_actual, list_predicted, k=10)

Recall@10:0.34
MRR@10:0.28735714285714287
MAP@10:0.26281415343915343
Average Score: 0.29672376543209883


In [ ]:
# original - No order nodes
from graphRAG.storage.chroma_db import average_four_metrics
list_actual = list(true_pos.values())
list_predicted = list(prediction_id.values())
average_four_metrics(list_actual, list_predicted, k=10)

Recall@10:0.335
MRR@10:0.36923809523809525
MAP@10:0.3198664021164021
Average Score: 0.3413681657848325


In [ ]:
# naive
from graphRAG.storage.chroma_db import average_four_metrics
list_actual = list(true_pos.values())
list_predicted = list(prediction_id.values())
average_four_metrics(list_actual, list_predicted, k=10)

Recall@10:0.44
MRR@10:0.6946031746031746
MAP@10:0.5247883597883598
Average Score: 0.5531305114638448


In [ ]:
# order + edges vs query
from graphRAG.storage.chroma_db import average_four_metrics
list_actual = list(true_pos.values())
list_predicted = list(prediction_id.values())
average_four_metrics(list_actual, list_predicted, k=10)

Recall@10:0.335
MRR@10:0.2865793650793651
MAP@10:0.2642757936507936
Average Score: 0.29528505291005286


In [15]:
# order + (edges vs query) * weight
from graphRAG.storage.chroma_db import average_four_metrics
list_actual = list(true_pos.values())
list_predicted = list(prediction_id.values())
average_four_metrics(list_actual, list_predicted, k=10)

Recall@10:0.335
MRR@10:0.2699920634920635
MAP@10:0.26080026455026456
Average Score: 0.28859744268077603


In [12]:
# (edges vs query) * weight
from graphRAG.storage.chroma_db import average_four_metrics
list_actual = list(true_pos.values())
list_predicted = list(prediction_id.values())
average_four_metrics(list_actual, list_predicted, k=10)

Recall@10:0.325
MRR@10:0.3295634920634921
MAP@10:0.30296560846560844
Average Score: 0.3191763668430336


In [29]:
# (edges vs query)
from graphRAG.storage.chroma_db import average_four_metrics
list_actual = list(true_pos.values())
list_predicted = list(prediction_id.values())
average_four_metrics(list_actual, list_predicted, k=10)

Recall@10:0.345
MRR@10:0.3626746031746032
MAP@10:0.32100925925925927
Average Score: 0.3428946208112875
